In [1]:
import momi
import itertools as it

In [8]:
!mkdir /nfs/users/nfs_j/jk21/lustre/scratch110/momi2/parse-vcftools-counts
!ls /nfs/users/nfs_j/jk21/lustre/scratch110/momi2

parse-vcftools-counts  read-plink


In [5]:
!ls /nfs/users/nfs_j/jk21/Documents/vervet/explore/raw

163_master_table_excl_library.csv
163_UG_ref3500_CAE10_snp_pass.vcf.gz
163_UG_ref3500_CAE10_snp_pass.vcf.gz.tbi
163_UG_ref3500_CAE10_snp_pass.vcf.gz.vcfidx
163_UG_ref3500_CAE11_snp_pass.vcf.gz
163_UG_ref3500_CAE11_snp_pass.vcf.gz.tbi
163_UG_ref3500_CAE11_snp_pass.vcf.gz.vcfidx
163_UG_ref3500_CAE12_snp_pass.vcf.gz
163_UG_ref3500_CAE12_snp_pass.vcf.gz.tbi
163_UG_ref3500_CAE12_snp_pass.vcf.gz.vcfidx
163_UG_ref3500_CAE13_snp_pass.vcf.gz
163_UG_ref3500_CAE13_snp_pass.vcf.gz.tbi
163_UG_ref3500_CAE13_snp_pass.vcf.gz.vcfidx
163_UG_ref3500_CAE14_snp_pass.vcf.gz
163_UG_ref3500_CAE14_snp_pass.vcf.gz.tbi
163_UG_ref3500_CAE14_snp_pass.vcf.gz.vcfidx
163_UG_ref3500_CAE15_snp_pass.vcf.gz
163_UG_ref3500_CAE15_snp_pass.vcf.gz.tbi
163_UG_ref3500_CAE15_snp_pass.vcf.gz.vcfidx
163_UG_ref3500_CAE16_snp_pass.vcf.gz
163_UG_ref3500_CAE16_snp_pass.vcf.gz.tbi
163_UG_ref3500_CAE16_snp_pass.vcf.gz.vcfidx
163_UG_ref3500_CAE17_snp_pass.vcf.gz
163_UG_ref3500_CAE17_snp_pass.vcf.gz.tbi
163_UG_ref3500_CAE17_snp_pass.vcf.

In [24]:
cmd = "vcf-concat {0} {0} {0}".format("~/Documents/vervet/explore/raw/163_UG_ref3500_CAE{}_snp_pass.vcf.gz")
cmd = cmd.format(26, 27, 28)
cmd += " | vcftools --vcf - --keep ~/Documents/vervet/explore/scratch/populations/{pop}.txt"
cmd += " --out ~/lustre/scratch110/momi2/parse-vcftools-counts/{pop}.counts"
cmd += " --counts2"
print(cmd)

vcf-concat ~/Documents/vervet/explore/raw/163_UG_ref3500_CAE26_snp_pass.vcf.gz ~/Documents/vervet/explore/raw/163_UG_ref3500_CAE27_snp_pass.vcf.gz ~/Documents/vervet/explore/raw/163_UG_ref3500_CAE28_snp_pass.vcf.gz | vcftools --vcf - --keep ~/Documents/vervet/explore/scratch/populations/{pop}.txt --out ~/lustre/scratch110/momi2/parse-vcftools-counts/{pop}.counts --counts2


In [25]:
gambia_cmd = cmd.format(pop="Gambia")
!{gambia_cmd}


VCFtools - 0.1.14
(C) Adam Auton and Anthony Marcketta 2009

Parameters as interpreted:
	--vcf [stdin]
	--keep /nfs/users/nfs_j/jk21/Documents/vervet/explore/scratch/populations/Gambia.txt
	--counts2
	--out /nfs/users/nfs_j/jk21/lustre/scratch110/momi2/parse-vcftools-counts/Gambia.counts

Keeping individuals in 'keep' list
After filtering, kept 22 out of 163 Individuals
Outputting Frequency Statistics...
After filtering, kept 3053808 out of a possible 3053808 Sites
Run Time = 176.00 seconds


In [26]:
barbados_cmd = cmd.format(pop="Barbados")
!{barbados_cmd}


VCFtools - 0.1.14
(C) Adam Auton and Anthony Marcketta 2009

Parameters as interpreted:
	--vcf [stdin]
	--keep /nfs/users/nfs_j/jk21/Documents/vervet/explore/scratch/populations/Barbados.txt
	--counts2
	--out /nfs/users/nfs_j/jk21/lustre/scratch110/momi2/parse-vcftools-counts/Barbados.counts

Keeping individuals in 'keep' list
After filtering, kept 5 out of 163 Individuals
Outputting Frequency Statistics...
After filtering, kept 3053808 out of a possible 3053808 Sites
Run Time = 188.00 seconds


In [2]:
def parse_vcftools_counts2(count_files_dict):
    def get_lines(count_fname):
        with open(count_fname) as f:
            header = next(f).split()
            expected_header = ["CHROM", "POS", "N_ALLELES", "N_CHR",  "{COUNT}"]
            if header != expected_header:
                raise IOError("Header {} does not match expected header {}".format(header, expected_header))
            for line in f:
                yield line.split()

    count_files_dict = {k: get_lines(v) for k,v in count_files_dict.items()}
    sampled_pops, lines_list = zip(*count_files_dict.items())
    def get_counts(lines_list):
        zipped_lines = it.zip_longest(*lines_list)
        for zline in zipped_lines:
            chrom, snp, n_alleles = zline[0][:3]
            if not all(l[:3] == zline[0][:3] for l in zline):
                raise IOError("Non-matching lines {}".format(zline))
            if int(n_alleles) != 2:
                continue
            ancestral = tuple(int(l[4]) for l in zline)
            derived = tuple(int(l[5]) for l in zline)
            assert (a+d == int(l[3]) for l,a,d in zip(ancestral, derived, zline))
            if all(a == 0 for a in ancestral) or all(d == 0 for d in derived):
                continue
            yield chrom, tuple(zip(ancestral, derived))
    lines_list = get_counts(lines_list)
    lines_list = it.groupby(lines_list, key=lambda x: x[0])

    return momi.seg_site_configs(sampled_pops, 
                                 ((config for c, config in chrom_lines) for chrom, chrom_lines in lines_list))    

In [3]:
count_files_dict = {"Barbados" : "/nfs/users/nfs_j/jk21/lustre/scratch110/momi2/parse-vcftools-counts/Barbados.counts.frq.count",
                   "Gambia" : "/nfs/users/nfs_j/jk21/lustre/scratch110/momi2/parse-vcftools-counts/Gambia.counts.frq.count"}

segsites = parse_vcftools_counts2(count_files_dict)

In [75]:
count_files_dict = {"Barbados" : "/nfs/users/nfs_j/jk21/lustre/scratch110/momi2/parse-vcftools-counts/Barbados.counts.frq.count",
                   "Gambia" : "/nfs/users/nfs_j/jk21/lustre/scratch110/momi2/parse-vcftools-counts/Gambia.counts.frq.count"}

def get_lines(count_fname):
    with open(count_fname) as f:
        header = next(f).split()
        expected_header = ["CHROM", "POS", "N_ALLELES", "N_CHR",  "{COUNT}"]
        if header != expected_header:
            raise IOError("Header {} does not match expected header {}".format(header, expected_header))
        for line in f:
            yield line.split()
            
count_files_dict = {k: get_lines(v) for k,v in count_files_dict.items()}
sampled_pops, lines_list = zip(*count_files_dict.items())
def get_counts(lines_list):
    zipped_lines = it.zip_longest(*lines_list)
    for zline in zipped_lines:
        chrom, snp, n_alleles = zline[0][:3]
        if not all(l[:3] == zline[0][:3] for l in zline):
            raise IOError("Non-matching lines {}".format(zline))
        if int(n_alleles) != 2:
            continue
        ancestral = tuple(int(l[4]) for l in zline)
        derived = tuple(int(l[5]) for l in zline)
        assert (a+d == int(l[3]) for l,a,d in zip(ancestral, derived, zline))
        if all(a == 0 for a in ancestral) or all(d == 0 for d in derived):
            continue
        yield chrom, tuple(zip(ancestral, derived))
lines_list = get_counts(lines_list)
lines_list = it.groupby(lines_list, key=lambda x: x[0])

segsites = momi.seg_site_configs(sampled_pops, ((config for c, config in chrom_lines) for chrom, chrom_lines in lines_list))

In [4]:
segsites

In [5]:
len(segsites)

3

In [6]:
segsites.sfs.n_snps()

1000548

In [7]:
len(segsites.configs)

8307

In [8]:
segsites[0][:10]

array([[[37,  1],
        [ 9,  1]],

       [[40,  2],
        [10,  0]],

       [[35,  1],
        [10,  0]],

       [[35,  1],
        [10,  0]],

       [[39,  1],
        [10,  0]],

       [[32,  4],
        [10,  0]],

       [[24, 18],
        [ 7,  3]],

       [[22, 22],
        [ 6,  4]],

       [[19, 23],
        [ 7,  3]],

       [[21, 23],
        [ 9,  1]]])

In [9]:
!head ~/lustre/scratch110/momi2/parse-vcftools-counts/Barbados.counts.frq.count
!head ~/lustre/scratch110/momi2/parse-vcftools-counts/Gambia.counts.frq.count

CHROM	POS	N_ALLELES	N_CHR	{COUNT}
CAE26	1000070	2	10	9	1
CAE26	1000482	2	10	10	0
CAE26	1000485	2	10	10	0
CAE26	1000493	2	10	10	0
CAE26	1000589	2	10	10	0
CAE26	1000594	2	10	10	0
CAE26	1001067	2	10	10	0
CAE26	1001085	2	10	10	0
CAE26	1001430	2	10	10	0
CHROM	POS	N_ALLELES	N_CHR	{COUNT}
CAE26	1000070	2	38	37	1
CAE26	1000482	2	42	42	0
CAE26	1000485	2	42	40	2
CAE26	1000493	2	42	42	0
CAE26	1000589	2	36	35	1
CAE26	1000594	2	36	35	1
CAE26	1001067	2	44	44	0
CAE26	1001085	2	44	44	0
CAE26	1001430	2	44	44	0


In [10]:
segsites.sampled_pops

('Gambia', 'Barbados')

In [11]:
!tail ~/lustre/scratch110/momi2/parse-vcftools-counts/Barbados.counts.frq.count
!tail ~/lustre/scratch110/momi2/parse-vcftools-counts/Gambia.counts.frq.count

CAE28	21530756	2	10	10	0
CAE28	21530894	2	8	8	0
CAE28	21530899	2	8	6	2
CAE28	21530909	2	8	8	0
CAE28	21530929	2	6	6	0
CAE28	21531017	2	8	8	0
CAE28	21531023	2	8	8	0
CAE28	21531055	2	6	6	0
CAE28	21531189	2	10	10	0
CAE28	21531196	2	10	10	0
CAE28	21530756	2	44	44	0
CAE28	21530894	2	44	44	0
CAE28	21530899	2	44	41	3
CAE28	21530909	2	42	42	0
CAE28	21530929	2	40	40	0
CAE28	21531017	2	44	39	5
CAE28	21531023	2	44	44	0
CAE28	21531055	2	42	42	0
CAE28	21531189	2	42	42	0
CAE28	21531196	2	42	42	0


In [12]:
segsites[-1][-10:]

array([[[41,  1],
        [10,  0]],

       [[21, 23],
        [ 9,  1]],

       [[37,  7],
        [10,  0]],

       [[40,  4],
        [10,  0]],

       [[36,  8],
        [10,  0]],

       [[30, 14],
        [10,  0]],

       [[26, 16],
        [ 9,  1]],

       [[39,  1],
        [10,  0]],

       [[41,  3],
        [ 6,  2]],

       [[39,  5],
        [ 8,  0]]])